# Script to get the jacobian matrix which under the .ipynb format

In [1]:
import sympy as sym
import numpy as np

In [2]:
# Direct state vector
x, y, z, w, l, h, v, a, yaw, sigma, lf_r, w_r, dt = sym.symbols("x y z w l h v a yaw sigma lf_r w_r dt")
next_x, next_y, next_z, next_w, next_h, next_l, next_v, next_a, next_yaw, next_sigma = sym.symbols("next_x next_y next_z next_w next_h next_l next_v next_a next_yaw next_sigma")

## Important formula

- $\beta$ (the slip angle between the velocity and heading of the object) 
    - $\beta(\tau) = tan^{-1}(\frac{l_r}{\gamma l} tan(\delta(\tau)))$
- $lr$ (the distance between the gravity center and the rear tire of the object)
    - $lr = l \cdot w_r \cdot (1 - lf_r)$
- $\eta$ (the angle between the velocity of the object and the $\textit{X-Axis}$ of the coordinate system)
    - $\eta = \theta + \beta$

Limited by the symbolic calculation ability of `sympy`, we manually simplified the calculation

In [3]:
# tmp state vector
beta, ry_rate, lf, lr = sym.symbols("beta ry_rate lf lr")
vyawt, ita, t = sym.symbols("vyawt ita t")
vyawt = yaw + beta + (v * sym.sin(beta) / lr) * ita

# Analytical solutions for displacements can also be obtained

In [4]:
next_x = x + sym.integrate(v * sym.cos(vyawt), (ita, 0, dt))
next_y = y + sym.integrate(v * sym.sin(vyawt), (ita, 0, dt))

In [5]:
next_x

x + Piecewise((dt*v*cos(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (dt*v*cos(yaw), Eq(beta, 0)), (-dt*v*cos(yaw), Eq(beta, pi)), (-lr*sin(beta + yaw)/sin(beta) + lr*sin(beta + dt*v*sin(beta)/lr + yaw)/sin(beta), True))

In [6]:
next_y

y + Piecewise((dt*v*sin(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (dt*v*sin(yaw), Eq(beta, 0)), (-dt*v*sin(yaw), Eq(beta, pi)), (lr*cos(beta + yaw)/sin(beta) - lr*cos(beta + dt*v*sin(beta)/lr + yaw)/sin(beta), True))

In [7]:
# state transition for heading yaw
next_yaw = yaw + beta + (v * sym.sin(beta) / lr) * dt

In [8]:
next_yaw

beta + dt*v*sin(beta)/lr + yaw

In [9]:
# state transition for constant state vector
next_z, next_w, next_h, next_l, next_a, next_sigma, next_v = z, w, h, l, 0, sigma, v

# Jacobian Matrix for state transition function

In [10]:
# large beta
funcs = sym.Matrix([next_x, next_y, next_z, next_w, next_l, next_h, next_v, next_a, next_yaw, next_sigma])
args = sym.Matrix([x, y, z, w, l, h, v, a, yaw, sigma])
res = funcs.jacobian(args)

In [11]:
res

Matrix([
[1, 0, 0, 0, 0, 0, Piecewise((dt*cos(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (dt*cos(yaw), Eq(beta, 0)), (-dt*cos(yaw), Eq(beta, pi)), (dt*cos(beta + dt*v*sin(beta)/lr + yaw), True)), 0, Piecewise((-dt*v*sin(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (-dt*v*sin(yaw), Eq(beta, 0)), (dt*v*sin(yaw), Eq(beta, pi)), (-lr*cos(beta + yaw)/sin(beta) + lr*cos(beta + dt*v*sin(beta)/lr + yaw)/sin(beta), True)), 0],
[0, 1, 0, 0, 0, 0, Piecewise((dt*sin(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (dt*sin(yaw), Eq(beta, 0)), (-dt*sin(yaw), Eq(beta, pi)), (dt*sin(beta + dt*v*sin(beta)/lr + yaw), True)), 0,  Piecewise((dt*v*cos(beta + yaw), Eq(v, 0) | (Eq(beta, 0) & Eq(v, 0)) | (Eq(beta, pi) & Eq(v, 0)) | (Eq(beta, 0) & Eq(beta, pi) & Eq(v, 0))), (dt*v*cos(yaw)

In [12]:
# tiny beta
funcs = sym.Matrix([x + (v * dt + a * dt * dt / 2) * sym.cos(yaw), 
                    y + (v * dt + a * dt * dt / 2) * sym.sin(yaw), 
                    next_z, next_w, next_l, next_h, 
                    v + a * dt, a, yaw, next_sigma])
res_zero = funcs.jacobian(args)

In [13]:
res_zero

Matrix([
[1, 0, 0, 0, 0, 0, dt*cos(yaw), dt**2*cos(yaw)/2, -(a*dt**2/2 + dt*v)*sin(yaw), 0],
[0, 1, 0, 0, 0, 0, dt*sin(yaw), dt**2*sin(yaw)/2,  (a*dt**2/2 + dt*v)*cos(yaw), 0],
[0, 0, 1, 0, 0, 0,           0,                0,                            0, 0],
[0, 0, 0, 1, 0, 0,           0,                0,                            0, 0],
[0, 0, 0, 0, 1, 0,           0,                0,                            0, 0],
[0, 0, 0, 0, 0, 1,           0,                0,                            0, 0],
[0, 0, 0, 0, 0, 0,           1,               dt,                            0, 0],
[0, 0, 0, 0, 0, 0,           0,                1,                            0, 0],
[0, 0, 0, 0, 0, 0,           0,                0,                            1, 0],
[0, 0, 0, 0, 0, 0,           0,                0,                            0, 1]])

# Jacobian Matrix for state transition function

In [14]:
mea_funcs = sym.Matrix([x - l * w_r * (0.5 - lf_r) * sym.cos(yaw), 
                        y - l * w_r * (0.5 - lf_r) * sym.sin(yaw), 
                        z, w, l, h, 
                        v * sym.cos(yaw + beta), v * sym.sin(yaw + beta), 
                        yaw])
mea_res = mea_funcs.jacobian(args)

In [15]:
mea_res, mea_res

(Matrix([
 [1, 0, 0, 0, -w_r*(0.5 - lf_r)*cos(yaw), 0,               0, 0,  l*w_r*(0.5 - lf_r)*sin(yaw), 0],
 [0, 1, 0, 0, -w_r*(0.5 - lf_r)*sin(yaw), 0,               0, 0, -l*w_r*(0.5 - lf_r)*cos(yaw), 0],
 [0, 0, 1, 0,                          0, 0,               0, 0,                            0, 0],
 [0, 0, 0, 1,                          0, 0,               0, 0,                            0, 0],
 [0, 0, 0, 0,                          1, 0,               0, 0,                            0, 0],
 [0, 0, 0, 0,                          0, 1,               0, 0,                            0, 0],
 [0, 0, 0, 0,                          0, 0, cos(beta + yaw), 0,           -v*sin(beta + yaw), 0],
 [0, 0, 0, 0,                          0, 0, sin(beta + yaw), 0,            v*cos(beta + yaw), 0],
 [0, 0, 0, 0,                          0, 0,               0, 0,                            1, 0]]),
 Matrix([
 [1, 0, 0, 0, -w_r*(0.5 - lf_r)*cos(yaw), 0,               0, 0,  l*w_r*(0.5 - lf_r)*si